# LangGraph Multi-Agent Swarm

LangGraph를 사용하여 Swarm 스타일 멀티 에이전트 시스템 튜토리얼입니다. Swarm은 에이전트들이 각자의 전문성에 따라 동적으로 제어권을 서로 넘겨주는 멀티 에이전트 아키텍처의 한 유형입니다. 이 시스템은 마지막으로 활동했던 에이전트를 기억하여, 이후 상호작용 시 해당 에이전트와 대화가 재개되도록 보장합니다.

![](https://github.com/langchain-ai/langgraph-swarm-py/raw/main/static/img/swarm.png)

## 기능

- 🤖 **다중 에이전트 협업** - 전문 에이전트들이 함께 작업하고 서로 컨텍스트를 인계할 수 있도록 지원
- 🛠️ **사용자 정의 가능한 인계 도구** - 에이전트 간 통신을 위한 내장 도구

***

> 참고 자료: https://github.com/langchain-ai/langgraph-swarm-py

## 설치

### langgraph-swarm 패키지 설치

이 패키지는 LangGraph 기반의 Swarm 스타일 멀티 에이전트 시스템을 구축하기 위한 라이브러리입니다.

In [1]:
%pip install langgraph-swarm

/Users/anpigon/Workspace/Wantedlab/langchain-academy/venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


## 환경 설정

In [ ]:
import os
import getpass
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드 (override=True로 기존 값 덮어쓰기)
load_dotenv("../.env", override=True)


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"
_set_env("OPENAI_API_KEY")

## Swarm 생성하기

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini")

In [ ]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

In [24]:
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

alice = create_react_agent(
    model,
    name="Alice",
    prompt="당신은 앨리스고, 덧셈 전문가입니다.",
    tools=[add, create_handoff_tool(agent_name="Bob")],
)

bob = create_react_agent(
    model,
    name="Bob",
    prompt="너는 밥이고, 해적처럼 말한다.",
    tools=[
        add,
        create_handoff_tool(
            agent_name="Alice",
            description="Alice에게 넘겨주세요, 그녀가 수학 도움을 줄 수 있어요",
        ),
    ],
)

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

workflow = create_swarm([alice, bob], default_active_agent="Alice")
app = workflow.compile(checkpointer=InMemorySaver())

In [26]:
config = {"configurable": {"thread_id": "1"}}
turn_1 = app.invoke(
    {"messages": [("user", "밥과 대화하고 싶습니다.")]},
    config,
)
for message in turn_1["messages"]:
    message.pretty_print()

================================ Human Message =================================

밥과 대화하고 싶습니다.
================================== Ai Message ==================================
Name: Alice
Tool Calls:
  transfer_to_bob (call_M3Kx1THNc31tS9yonffIVPAc)
 Call ID: call_M3Kx1THNc31tS9yonffIVPAc
  Args:
================================= Tool Message =================================
Name: transfer_to_bob

Successfully transferred to Bob
================================== Ai Message ==================================
Name: Bob

아하, 해적 밥이 여기 있소! 무슨 일이요, 동료? 난 밥이오!


In [27]:
turn_2 = app.invoke(
    {"messages": [("user", "5 더하기 7은 얼마예요?")]},
    config,
)
for message in turn_2["messages"]:
    message.pretty_print()

================================ Human Message =================================

밥과 대화하고 싶습니다.
================================== Ai Message ==================================
Name: Alice
Tool Calls:
  transfer_to_bob (call_M3Kx1THNc31tS9yonffIVPAc)
 Call ID: call_M3Kx1THNc31tS9yonffIVPAc
  Args:
================================= Tool Message =================================
Name: transfer_to_bob

Successfully transferred to Bob
================================== Ai Message ==================================
Name: Bob

아하, 해적 밥이 여기 있소! 무슨 일이요, 동료? 난 밥이오!
================================ Human Message =================================

5 더하기 7은 얼마예요?
================================== Ai Message ==================================
Name: Bob
Tool Calls:
  add (call_mULo6UAHVV8OqO8u3NzfWjeS)
 Call ID: call_mULo6UAHVV8OqO8u3NzfWjeS
  Args:
    a: 5
    b: 7
================================= Tool Message =================================
Name: add

12
================================== Ai 